# Demo Notebook for Creating a Pitch by Pitch Report with Reboot Motion Metrics

Run the cells in order, making sure to enter AWS credentials in the cell when prompted


In [ ]:
#@title Install Python Package

!pip install -q git+https://github.com/RebootMotion/colab-notebook-utils.git@v1.0.0#egg=colab_notebook_utils

In [ ]:
import boto3
import colab_notebook_utils as utils
import matplotlib.pyplot as plt
import os
import pandas as pd

from colab_notebook_utils import S3Metadata, MocapType, MovementType, Handedness, FileType, PlayerMetadata

In [ ]:
#@title Org and AWS Credentials

from dotenv import load_dotenv

load_dotenv()

if 'AWS_ACCESS_KEY_ID' not in os.environ:
    from getpass import getpass

    os.environ['ORG_ID'] = getpass('Enter reboot-motion org_id here:')
    os.environ['AWS_ACCESS_KEY_ID'] = getpass('Enter AWS_ACCESS_KEY_ID here:')
    os.environ['AWS_SECRET_ACCESS_KEY'] = getpass('Enter SECRET_ACCESS_KEY here:')
    os.environ['AWS_DEFAULT_REGION'] = getpass('Enter AWS_DEFAULT_REGION here:')

boto3_session = boto3.session.Session(
    aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
    region_name=os.environ['AWS_DEFAULT_REGION']
    )

print('Current Boto3 Session:')
print(boto3_session)

In [ ]:
#@title Set S3 File Info

s3_metadata = S3Metadata(
    org_id=os.environ['ORG_ID'],
    mocap_type=MocapType.HAWKEYE_HFR,
    movement_type=MovementType.BASEBALL_PITCHING,
    handedness=Handedness.LEFT,
    file_type=FileType.METRICS_BASEBALL_PITCHING_V2
)

s3_df = utils.download_s3_summary_df(s3_metadata)

In [ ]:
#@title Set Primary Analysis Segment Info

primary_segment_label = '2023-04-13'

primary_analysis_segment = PlayerMetadata(
    mlbam_player_ids=['605488'],
    session_dates=None,
    game_pks=None, # None defaults to all
    session_date_start = '2023-04-13',
    session_date_end = None,
    year = None,
    mlb_play_guid=None, # set the mlb play GUID for the skeleton animation; None defaults to the first play
    s3_metadata=s3_metadata,
)

primary_segment_summary_df = utils.filter_s3_summary_df(primary_analysis_segment, s3_df)

In [ ]:
#@title Add Movement Num and S3 Key to Primary DataFrame to Enable Sorting

prim_available_s3_keys = utils.list_available_s3_keys(os.environ['ORG_ID'], primary_segment_summary_df)

primary_segment_data_df = utils.load_games_to_df_from_s3_paths(primary_segment_summary_df['s3_path_delivery'].tolist())

primary_segment_data_df = utils.merge_data_df_with_s3_keys(primary_segment_data_df, prim_available_s3_keys).sort_values(by=['session_date', 'movement_num'])

In [ ]:
#@title Optional - Uncomment below and set Comparison Analysis Segment Inputs

comparison_segment_label = 'Early 2023'

comparison_s3_metadata = s3_metadata

comparison_analysis_segment = PlayerMetadata(
    mlbam_player_ids=['605488'],
    session_dates=None, # None defaults to all
    game_pks=None, # None defaults to all,
    session_date_start = None,
    session_date_end = '2023-04-12',
    year = None,
    mlb_play_guid=None, # set the mlb play GUID for the skeleton animation; None defaults to the first play
    s3_metadata=comparison_s3_metadata,
)

comparison_segment_summary_df = utils.filter_s3_summary_df(comparison_analysis_segment, s3_df)

comp_available_s3_keys = utils.list_available_s3_keys(os.environ['ORG_ID'], comparison_segment_summary_df)

comparison_segment_data_df = utils.load_games_to_df_from_s3_paths(comparison_segment_summary_df['s3_path_delivery'].tolist())

comparison_segment_data_df = utils.merge_data_df_with_s3_keys(comparison_segment_data_df , comp_available_s3_keys).sort_values(by=['session_date', 'movement_num'])

In [ ]:
comparison_segment_data_df['segment_type'] = 'comparison'

primary_segment_data_df['segment_type'] = 'primary'

analysis_df = pd.concat([comparison_segment_data_df, primary_segment_data_df]).sort_values(by=['session_date', 'movement_num']).reset_index(drop=True)

In [ ]:
print('Available metrics:')
print()
print(list(analysis_df))

In [ ]:
body_segments = ['rear_leg', 'lead_leg', 'torso', 'pitch_up_arm', 'pitch_forearm', 'pitch_hand']
metric_categories = ['proj_max']
metrics = [f"{body_seg}_{metric_cat}" for body_seg in body_segments for metric_cat in metric_categories]

# body_joints = ['rear_hip_ir', 'lead_knee_flex', 'lumbar_ext', 'spine_rot', 'pitch_shoulder_horz', 'pitch_shoulder_rot', 'pitch_elbow_flex']
# metric_categories = ['range_full_min', 'range_full_max', 'range_min', 'range_max', 'range_norm]
# metrics = [f"{body_joint}_{metric_cat}" for body_joint in body_joints for metric_cat in metric_categories]

# metrics = ['lower_half_to_torso_proj', 'torso_to_up_arm_proj', 'up_arm_to_pitch_hand_proj']

comparison_color = 'green'
primary_color = 'red'

rolling_window = 10

figs = []

comparison = analysis_df.loc[analysis_df['segment_type'] == 'comparison']
x_comp = comparison.index.tolist()

primary = analysis_df.loc[analysis_df['segment_type'] == 'primary']
x_prim = primary.index.tolist()

for metric in metrics:

    y_comp = comparison[metric].rolling(window=rolling_window).mean()
    y_comp_std = comparison[metric].rolling(window=rolling_window).std()

    y_comp_lo = y_comp - y_comp_std
    y_comp_hi = y_comp + y_comp_std

    y_prim = primary[metric].rolling(window=rolling_window).mean()
    y_prim_std = primary[metric].rolling(window=rolling_window).std()

    y_prim_lo = y_prim - y_prim_std
    y_prim_hi = y_prim + y_prim_std

    fig = plt.figure()

    plt.fill_between(x_comp, y_comp_lo, y_comp_hi, color=comparison_color, alpha=0.2)
    plt.plot(x_comp, y_comp, comparison_color)

    plt.fill_between(x_prim, y_prim_lo, y_prim_hi, color=primary_color, alpha=0.2)
    plt.plot(x_prim, y_prim, primary_color)

    plt.ylabel('metric value')
    plt.xlabel('season pitch count')
    plt.title(metric)

    plt.grid()

    figs.append(fig)

    plt.show()

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

pdf_file_name = 'analysis_pitch_by_pitch.pdf'

pdf_analysis = PdfPages(pdf_file_name)

for fig in figs:

    pdf_analysis.savefig(fig)

pdf_analysis.close()

print('Saved plots to', pdf_file_name)